In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import statsmodels.api as sm


In [2]:
ls data/

ls: data/: No such file or directory


In [3]:
ls

usdc_analysis.ipynb           usdc_eth_analysis.ipynb
usdc_eth_180d_analysis.ipynb


In [4]:
pwd


'/Users/manarelkhateb/Documents/Documents - Manar’s MacBook Pro/Resume /projects/stablecoin-onchain-analysis/notebooks'

In [5]:
ls ../data/

usdc_raw_180d.csv


In [6]:
df = pd.read_csv("../data/usdc_raw_180d.csv")
df.shape

(190000, 21)

In [8]:
df.head()

,blockNumber,timeStamp,hash,nonce,blockHash,from,contractAddress,to,value,tokenName,...,tokenDecimal,transactionIndex,gas,gasPrice,gasUsed,cumulativeGasUsed,input,methodId,functionName,confirmations
0,24241760,1768499963,0xb1359cc425b06b330cdd5ff040127108bb40b34de0bd...,826,0x077a6d8dc38278dc569690a6b31bf063418e90728439...,0xbebc44782c7db0a1a60cb6fe97d0b483032ff1c7,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xd2b37ade14708bf18904047b1e31f8166d39612b,164967975,USDC,...,6,1,562860,261785959,367995,388995,deprecated,0x5f3bd1c8,NaN,7180
1,24241760,1768499963,0xb1359cc425b06b330cdd5ff040127108bb40b34de0bd...,826,0x077a6d8dc38278dc569690a6b31bf063418e90728439...,0xd2b37ade14708bf18904047b1e31f8166d39612b,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xde7259893af7cdbc9fd806c6ba61d22d581d5667,575747,USDC,...,6,1,562860,261785959,367995,388995,deprecated,0x5f3bd1c8,NaN,7180
2,24241760,1768499963,0xb1359cc425b06b330cdd5ff040127108bb40b34de0bd...,826,0x077a6d8dc38278dc569690a6b31bf063418e90728439...,0xd2b37ade14708bf18904047b1e31f8166d39612b,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x25a04bc7379faabc0be0421da74191ea51713c8c,164392228,USDC,...,6,1,562860,261785959,367995,388995,deprecated,0x5f3bd1c8,NaN,7180
3,24241760,1768499963,0x8a5744062067e7241f3e4614b4c36ce6751752a726fc...,123839,0x077a6d8dc38278dc569690a6b31bf063418e90728439...,0x000000000004444c5dc75cb358380d2e3de08a90,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x0000000aa232009084bd71a5797d089aa4edfad4,10591056670,USDC,...,6,2,493994,179515959,287101,676096,deprecated,0x09c5eabe,execute(bytes data),7180
4,24241760,1768499963,0x8a5744062067e7241f3e4614b4c36ce6751752a726fc...,123839,0x077a6d8dc38278dc569690a6b31bf063418e90728439...,0x0000000aa232009084bd71a5797d089aa4edfad4,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x73d53553ee552c1f2a9722e6407d43e41e19593f,10577942120,USDC,...,6,2,493994,179515959,287101,676096,deprecated,0x09c5eabe,execute(bytes data),7180


In [9]:
# Convert timestamp to datetime
df["timeStamp"] = pd.to_datetime(df["timeStamp"], unit="s")

# Create a date column (no time, just date)
df["date"] = df["timeStamp"].dt.date

# Convert USDC value (6 decimals)
df["usdc_amount"] = df["value"].astype(float) / 1_000_000

df[["timeStamp", "date", "usdc_amount"]].head()


,timeStamp,date,usdc_amount
0,2026-01-15 17:59:23,2026-01-15,164.967975
1,2026-01-15 17:59:23,2026-01-15,0.575747
2,2026-01-15 17:59:23,2026-01-15,164.392228
3,2026-01-15 17:59:23,2026-01-15,10591.056670
4,2026-01-15 17:59:23,2026-01-15,10577.942120


In [10]:
df["date"].min(), df["date"].max()


(datetime.date(2025, 12, 17), datetime.date(2026, 1, 15))

In [11]:
daily_usdc = (
    df.groupby("date")["usdc_amount"]
      .sum()
      .reset_index()
)

daily_usdc.shape


(19, 2)